In [1]:
import numpy as np
from intent_utils import *
import matplotlib.pyplot as plt
from keras.models import load_model
import time
import os

%matplotlib inline

Using TensorFlow backend.


In [2]:
X_train, Y_train = read_csv('data/train_intent.csv')
X_test, Y_test = read_csv('data/test_intent.csv')

In [3]:
#maxLen = len(max(X_train, key=len).split())
tokenized_sentences = [sentence.split(" ") for sentence in X_train]
longest_sen = max(tokenized_sentences, key=len)
maxLen = len(longest_sen)
print(maxLen)

17


In [4]:
index = 1
print(X_train[index], label_to_class(Y_train[index]))

Good Afternoon Call_Opening


To get our labels into a format suitable for training a softmax classifier, lets convert $Y$ from its current shape  current shape $(m, 1)$ into a "one-hot representation" $(m, 5)$, where each row is a one-hot vector giving the label of one example.

In [5]:
Y_oh_train = convert_to_one_hot(Y_train, C = 15)
Y_oh_test = convert_to_one_hot(Y_test, C = 15)

Let's see what `convert_to_one_hot()` did.

In [6]:
index = 50
print(Y_train[index], "is converted into one hot", Y_oh_train[index])

6 is converted into one hot [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]


All the data is now ready to be fed into the Intent class model. Let's implement the model!

### Time to load the Glove

We will use pretrained 50-dimensional GloVe embeddings.

In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

In [8]:
# Adding unknown word details
word_to_vec_map["<UNK>"] = np.zeros((50))
index_to_word[400001] = "<UNK>"
word_to_index["<UNK>"] = 400001


- `word_to_index`: dictionary mapping from words to their indices in the vocabulary (400,001 words, with the valid indices ranging from 0 to 400,000)
- `index_to_word`: dictionary mapping from indices to their corresponding words in the vocabulary
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.


## INTENT CLASS: Using LSTMs in Keras: 

In [9]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

In [10]:

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()`. 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros(((m,max_len)))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =X[i].lower().split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            try:
                embedding_vector = word_to_index[w]     
            except KeyError:
                embedding_vector = word_to_index["<UNK>"]      #Its an unknown word
           
            if embedding_vector is not None:
                # X_indices[i, j] = word_to_index[w]
                X_indices[i, j] = embedding_vector
                # Increment j to j + 1
                j = j +1
                if j>=max_len:
                    break
            
    ### END CODE HERE ###
    
    return X_indices

In [11]:
word_to_vec_map['<UNK>']
print(index_to_word[400001])

<UNK>


Testing the  `sentences_to_indices()` function

In [12]:
X1 = np.array(["Municipal corporation", "May i know your salary", "Can i setup a call"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 30)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['Municipal corporation' 'May i know your salary' 'Can i setup a call']
X1_indices = [[ 252973.  109427.       0.       0.       0.       0.       0.       0.
        0.       0.       0.       0.       0.       0.       0.       0.
        0.       0.       0.       0.       0.       0.       0.       0.
        0.       0.       0.       0.       0.       0.]
 [ 237383.  185457.  209645.  394565.  316259.       0.       0.       0.
        0.       0.       0.       0.       0.       0.       0.       0.
        0.       0.       0.       0.       0.       0.       0.       0.
        0.       0.       0.       0.       0.       0.]
 [  90548.  185457.  326012.   43010.   89824.       0.       0.       0.
        0.       0.       0.       0.       0.       0.       0.       0.
        0.       0.       0.       0.       0.       0.       0.       0.
        0.       0.       0.       0.       0.       0.]]


In [13]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["good"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len,
                            emb_dim,
                            weights=[emb_matrix],
                            trainable=True)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [14]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.3403


**Expected Output**:

<table>
    <tr>
        <td>
            **weights[0][1][3] =**
        </td>
        <td>
           -0.3403
        </td>
    </tr>
</table>

In [15]:
def Sentence_Intent(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Intent class model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)  
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences = True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences = False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(15,activation='softmax')(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices,X)
    
    ### END CODE HERE ###
    
    return model

In [16]:
model = Sentence_Intent((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 17)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 17, 50)            20000100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 17, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 17, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                1935      
__________

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

It's time to train the model. The `model` takes as input an array of shape (`m`, `max_len`) and outputs probability vectors of shape (`m`, `number of classes`).

In [18]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 15)

In [19]:
model.fit(X_train_indices, Y_train_oh, epochs = 200, batch_size = 32, shuffle=True)

Epoch 1/200
121/121 [==============================] - 1s 12ms/step - loss: 2.7064 - acc: 0.1157
Epoch 2/200
121/121 [==============================] - 1s 9ms/step - loss: 2.6947 - acc: 0.1570
Epoch 3/200
121/121 [==============================] - 1s 9ms/step - loss: 2.6740 - acc: 0.1488
Epoch 4/200
121/121 [==============================] - 1s 9ms/step - loss: 2.6610 - acc: 0.1322
Epoch 5/200
121/121 [==============================] - 1s 9ms/step - loss: 2.6626 - acc: 0.1322
Epoch 6/200
121/121 [==============================] - 1s 9ms/step - loss: 2.6302 - acc: 0.1736
Epoch 7/200
121/121 [==============================] - 1s 9ms/step - loss: 2.5810 - acc: 0.3306
Epoch 8/200
121/121 [==============================] - 1s 9ms/step - loss: 2.5472 - acc: 0.3554
Epoch 9/200
121/121 [==============================] - 1s 9ms/step - loss: 2.4866 - acc: 0.3884
Epoch 10/200
121/121 [==============================] - 1s 9ms/step - loss: 2.4481 - acc: 0.4215
Epoch 11/200
121/121 [================

121/121 [==============================] - 1s 9ms/step - loss: 1.9178 - acc: 0.9008
Epoch 86/200
121/121 [==============================] - 1s 9ms/step - loss: 1.9195 - acc: 0.9008
Epoch 87/200
121/121 [==============================] - 1s 9ms/step - loss: 1.9180 - acc: 0.9008
Epoch 88/200
121/121 [==============================] - 1s 9ms/step - loss: 1.9184 - acc: 0.9008
Epoch 89/200
121/121 [==============================] - 1s 9ms/step - loss: 1.9171 - acc: 0.9008
Epoch 90/200
121/121 [==============================] - 1s 9ms/step - loss: 1.9177 - acc: 0.9008
Epoch 91/200
121/121 [==============================] - 1s 9ms/step - loss: 1.9173 - acc: 0.9008
Epoch 92/200
121/121 [==============================] - 1s 9ms/step - loss: 1.9161 - acc: 0.9008
Epoch 93/200
121/121 [==============================] - 1s 9ms/step - loss: 1.9173 - acc: 0.9008
Epoch 94/200
121/121 [==============================] - 1s 9ms/step - loss: 1.9157 - acc: 0.9008
Epoch 95/200
121/121 [=====================

121/121 [==============================] - 1s 9ms/step - loss: 1.8269 - acc: 0.9917
Epoch 169/200
121/121 [==============================] - 1s 9ms/step - loss: 1.8262 - acc: 0.9917
Epoch 170/200
121/121 [==============================] - 1s 9ms/step - loss: 1.8258 - acc: 0.9917
Epoch 171/200
121/121 [==============================] - 1s 9ms/step - loss: 1.8258 - acc: 0.9917
Epoch 172/200
121/121 [==============================] - 1s 9ms/step - loss: 1.8261 - acc: 0.9917
Epoch 173/200
121/121 [==============================] - 1s 9ms/step - loss: 1.8257 - acc: 0.9917
Epoch 174/200
121/121 [==============================] - 1s 9ms/step - loss: 1.8269 - acc: 0.9917
Epoch 175/200
121/121 [==============================] - 1s 9ms/step - loss: 1.8257 - acc: 0.9917
Epoch 176/200
121/121 [==============================] - 1s 9ms/step - loss: 1.8260 - acc: 0.9917
Epoch 177/200
121/121 [==============================] - 1s 9ms/step - loss: 1.8258 - acc: 0.9917
Epoch 178/200
121/121 [===========

Ideally model should perform close to **100% accuracy** on the training set.  Run the following cell to evaluate your model on the test set. 

In [20]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 15)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

127/127 [==============================] - 0s 962us/step

Test accuracy =  0.889763781405


If the accuracy is more than 80% Save it

In [21]:
if acc > 0.80: # We will only save models if the accuracy goes beyond 80%
    timestr = time.strftime("%Y%m%d-%H%M%S")
    model_file_name = "Intent_class_model_%s_%s.h5" %(timestr,str(acc))
    model_folder_name = 'model'
    model.save(os.path.join(model_folder_name,model_file_name))  # creates a HDF5 file 'my_model.h5'

Accuracy between 80% and 95% should be OK for now. Run the cell below to see the mislabelled examples. 

In [22]:
# This code allows you to see the mislabelled examples
C = 15
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)

pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected class:'+ label_to_class(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_class(num).strip())

Expected class:Profession_Qualification prediction: Sir May I know Are you salaried or self employedCall_Closure_Permission
Expected class:Profession_Qualification prediction: Sir may I know are you a business man or doing a jobCustomer_VOC
Expected class:Profession_Qualification prediction: Sir may I know your occupationCustomer_VOC
Expected class:Income_Qualification_2 prediction: OK Sir May I know What is the total EMI you are paying on your previous loansCustomer_VOC
Expected class:Family_Income_Qualification prediction: Madam may I know if you have any other source of incomeIncome_Qualification_2
Expected class:Family_Income_Qualification prediction: Madam May I know if there is any other family member who is earningCustomer_VOC
Expected class:Property_Qualification_3 prediction: Sir may I know if you Have you selected the propertyCustomer_VOC
Expected class:Property_Qualification_3 prediction: Sir may I know if you Have you identified the propertyCustomer_VOC
Expected class:Prope

Now you can try it on your own example. Write your own sentence below. 

In [23]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.  
x_test = np.array(['morning sir'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_class(np.argmax(model.predict(X_test_indices))))

morning sir Call_Opening
